In [1]:
from src.model_db.tiny_model_db import TinyModelDB

In [2]:
env_name = 'Humanoid-v4'    
policy_db = TinyModelDB(base_path=f'saved_models/rl/{env_name}')

In [19]:
for policy_entry in sorted(policy_db.all_entries(), key=lambda p: p['model_info']['score'], reverse=True):
    print(f"id = {policy_entry['model_id']}, score = {policy_entry['model_info']['score']:6.4f}, parent_id = {str(policy_entry['parent_model_id'])}")

id = 2024-04-30_22.09.28, score = 1.0121, parent_id = None
id = 2024-04-30_22.08.06, score = 0.2851, parent_id = None


In [14]:
model_id = '2024-04-30_22.08.06'

def print_dict(d: dict):
    for key, value in d.items():
        if isinstance(value, dict):
            print(f"↓↓↓  {key:^15}  ↓↓↓")
            print_dict(value)
            print(f"↑↑↑  {key:^15}  ↑↑↑\n\n")
        else:
            print(f"======  {key:^15}  ======\n{value}\n")

print_dict(policy_db.fetch_entry(model_id))

======     model_id      ======
2024-04-30_22.08.06

======  parent_model_id  ======
None

======  state_dict_path  ======
saved_models/rl/Humanoid-v4\2024-04-30_22.08.06--state_dict.pth

↓↓↓    model_info     ↓↓↓
======       score       ======
0.28508483138860485

======  wrap_env_function  ======
def wrap_env(_env: Env):
    # _env = NormalizeRewardWrapper(_env, gamma=gamma)
    # _env = TransformObservation(_env, lambda _obs: _obs / 255)
    _env = TransformReward(_env, lambda _reward: 0.01 * _reward) 
    _env = RescaleAction(_env, min_action=-1.0, max_action=1.0)
    _env = ClipAction(_env)
    return _env


↑↑↑    model_info     ↑↑↑


======   init_function   ======
def init_policy():
    class A2CNetwork(nn.Module):

        def __init__(self):
            super().__init__()
            
            in_size = 376
            shared_out_sizes = []
            actor_out_sizes = [512, 512, 256, 256, 256, 256, 256, 256, 17]
            critic_out_sizes = [512, 512, 256, 256, 256, 1

In [3]:
policy_db.delete_entry('2024-04-30_09.24.10', delete_state_dict=True)

ValueError: Model id 2024-04-30_09.24.10 not found